In [ ]:
import os
import subprocess
import sys

# Save current working directory
cwd = os.getcwd()

# Clone the repo if it doesn't exist
if not os.path.exists("DiffSynth-Studio"):
    subprocess.run(["git", "clone", "https://github.com/modelscope/DiffSynth-Studio.git"])

# Change into the repo directory
os.chdir("DiffSynth-Studio")

# Install in editable mode using the current Python environment
subprocess.run([sys.executable, "-m", "pip", "install", "-e", "."])

# Go back to original directory
os.chdir(cwd)

print("✅ DiffSynth-Studio installed and returned to original directory.")

In [ ]:
!pip install accelerate --upgrade
!pip install -e DiffSynth-Studio

In [ ]:
!accelerate launch DiffSynth-Studio/examples/qwen_image/model_training/train.py \
  --dataset_base_path data/ranveer_singh \
  --dataset_metadata_path data/ranveer_singh/metadata.json \
  --data_file_keys "image" \
  --max_pixels 1048576 \
  --model_id_with_origin_paths "Qwen/Qwen-Image-Edit-2509:text_encoder/model*.safetensors,Qwen/Qwen-Image-Edit-2509:vae/diffusion_pytorch_model.safetensors" \
  --output_path "./models/ranveer_cache" \
  --task data_process \
  --dataset_num_workers 8

In [ ]:
import torch,gc
torch.cuda.empty_cache()
gc.collect()

import os  
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

!accelerate launch DiffSynth-Studio/examples/qwen_image/model_training/train.py \
  --dataset_base_path models/ranveer_cache \
  --max_pixels 786432 \
  --dataset_repeat 15 \
  --model_id_with_origin_paths "Qwen/Qwen-Image-Edit-2509:transformer/diffusion_pytorch_model*.safetensors" \
  --learning_rate 1e-4 \
  --num_epochs 5 \
  --remove_prefix_in_ckpt "pipe.dit." \
  --output_path "./models/train/ranveer_lora" \
  --lora_base_model "dit" \
  --lora_target_modules "to_q,to_k,to_v,add_q_proj,add_k_proj,add_v_proj,to_out.0,to_add_out,img_mlp.net.2,img_mod.1,txt_mlp.net.2,txt_mod.1" \
  --lora_rank 64 \
  --use_gradient_checkpointing \
  --use_gradient_checkpointing_offload \
  --gradient_accumulation_steps 8 \
  --dataset_num_workers 8 \
  --find_unused_parameters \
  --enable_fp8_training \
  --task sft \
  --save_steps 2000

In [ ]:
!cd DiffSynth-Studio && pip install -e .
!pip install -e DiffSynth-Studio

In [ ]:
!pip install hf_transfer

In [ ]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

import sys, os
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '0'  
sys.path.append(os.path.abspath("./DiffSynth-Studio"))
from diffsynth.pipelines.qwen_image import QwenImagePipeline, ModelConfig  
from PIL import Image  
import torch  
  
# Load the base model with processor from ModelScope  
pipe = QwenImagePipeline.from_pretrained(  
    torch_dtype=torch.bfloat16,  
    device="cuda",  
    model_configs=[  
        ModelConfig(model_id="Qwen/Qwen-Image-Edit-2509", origin_file_pattern="transformer/diffusion_pytorch_model*.safetensors"),  
        ModelConfig(model_id="Qwen/Qwen-Image-Edit-2509", origin_file_pattern="text_encoder/model*.safetensors"),  
        ModelConfig(model_id="Qwen/Qwen-Image-Edit-2509", origin_file_pattern="vae/diffusion_pytorch_model.safetensors"),  
    ],  
    tokenizer_config=ModelConfig(model_id="Qwen/Qwen-Image-Edit-2509", origin_file_pattern="tokenizer/"),  
    processor_config=None  
)  
  
# Load processor from a Qwen2.5-VL model that has processor files  
from transformers import Qwen2VLProcessor  
pipe.processor = Qwen2VLProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct", trust_remote_code=True) 
  
# # Load your trained LoRA  
# pipe.load_lora(pipe.dit, "./models/train/ranveer_lora/step-100.safetensors")  
  
# # Edit an image  
# image = pipe(  
#     prompt="Replace the person with Ranveer Singh",  
#     edit_image=Image.open("images.jpg"),  
#     seed=42,  
#     num_inference_steps=40  
# )    
# image.save("test_output1.jpg")


# Path to LoRA checkpoints
lora_dir = "./models/train/ranveer_lora"

# Load edit image once
edit_image = Image.open("rahul.jpeg")

# Loop through all .safetensors files in lora_dir
for file_name in sorted(os.listdir(lora_dir)):
    if file_name.endswith(".safetensors"):
        lora_path = os.path.join(lora_dir, file_name)
        print(f"🔹 Testing LoRA: {file_name}")

        # Load LoRA weights
        pipe.load_lora(pipe.dit, lora_path, alpha=1.5)

        output = pipe(
            prompt="Replace the person face with Ranveer Singh",
            edit_image=edit_image,
            cfg_scale=9.0,
            seed=42,
            num_inference_steps=60
        )

        # Save with unique filename
        output_path = f"rahul_outputs/{os.path.splitext(file_name)[0]}_output.jpg"
        os.makedirs("rahul_outputs", exist_ok=True)
        output.save(output_path)

        print(f"✅ Saved: {output_path}")

In [ ]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

import sys, os
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '0'  
from diffsynth.pipelines.qwen_image import QwenImagePipeline, ModelConfig  
from PIL import Image  
import torch  
  
# Load base Qwen-Image model  
pipe = QwenImagePipeline.from_pretrained(  
    torch_dtype=torch.bfloat16,  
    device="cuda",  
    model_configs=[  
        ModelConfig(model_id="Qwen/Qwen-Image",   
                   origin_file_pattern="transformer/diffusion_pytorch_model*.safetensors"),  
        ModelConfig(model_id="Qwen/Qwen-Image",   
                   origin_file_pattern="text_encoder/model*.safetensors"),  
        ModelConfig(model_id="Qwen/Qwen-Image",   
                   origin_file_pattern="vae/diffusion_pytorch_model.safetensors"),  
    ],  
    tokenizer_config=ModelConfig(model_id="Qwen/Qwen-Image",   
                                 origin_file_pattern="tokenizer/"),  
    # Note: No processor_config needed for base model  
)  
  
# Load your trained LoRA  
pipe.load_lora(pipe.dit, "./models/train/ranveer_lora/step-4000.safetensors", alpha=1.5)  
  
# Generate new image  
output = pipe(  
    prompt="Ranveer Singh in a mahabharat",  
    cfg_scale=7.5,  # Base model uses default cfg_scale=4.0  
    seed=42,  
    num_inference_steps=40,  # Base model uses default 30 steps  
    height=1328,  
    width=1328  
)  
output.save("ranveer_singh_in_mahabharat.jpg")
